In [1]:
import pandas as pd
import numpy as np

In [2]:
# Chargement du dataset Kaggle
df_kaggle = pd.read_csv("../../Data/2025/atp_tennis_2025.csv")

# Conversion de la colonne de date au bon format
df_kaggle["Date"] = pd.to_datetime(df_kaggle["Date"], errors='coerce')

# Filtrage sur l'année 2025 uniquement
df_2025 = df_kaggle[df_kaggle["Date"].dt.year == 2025].copy()

print(f"{len(df_2025)} matches found for 2025")
df_2025.head()

958 matches found for 2025


,Tournament,Date,Series,Court,Surface,Round,Best of,Player_1,Player_2,Winner,Rank_1,Rank_2,Pts_1,Pts_2,Odd_1,Odd_2,Score
64181,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Nishioka Y.,Lehecka J.,Lehecka J.,69,28,776,1660,3.20,1.36,2-6 7-6 4-6
64182,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Jarry N.,Bonzi B.,Jarry N.,35,75,1370,730,2.63,1.50,6-2 6-3
64183,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Vukic A.,Dimitrov G.,Dimitrov G.,68,10,778,3350,4.50,1.20,2-6 6-7
64184,Brisbane International,2025-01-01,ATP250,Outdoor,Hard,2nd Round,3,Thompson J.,Michelsen A.,Thompson J.,26,41,1745,1245,1.67,2.20,7-5 6-3
64185,Brisbane International,2025-01-02,ATP250,Outdoor,Hard,2nd Round,3,Lajovic D.,Mensik J.,Mensik J.,81,48,710,1136,4.00,1.25,3-6 2-6


In [ ]:
# Dataset Jeff Sackmann pour structure de référence
df_jeff = pd.read_csv("../../Data/atp/atp_matches_2024.csv")
df_jeff.head()


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
0,2024-0339,Brisbane,Hard,32,A,20240101,300,105777,2.0,NaN,...,58.0,44.0,16.0,11.0,8.0,9.0,14.0,2570.0,8.0,3660.0
1,2024-0339,Brisbane,Hard,32,A,20240101,299,208029,1.0,NaN,...,35.0,31.0,10.0,11.0,5.0,7.0,8.0,3660.0,39.0,1122.0
2,2024-0339,Brisbane,Hard,32,A,20240101,298,105777,2.0,NaN,...,39.0,24.0,14.0,10.0,5.0,7.0,14.0,2570.0,55.0,902.0
3,2024-0339,Brisbane,Hard,32,A,20240101,297,208029,1.0,NaN,...,51.0,31.0,16.0,10.0,3.0,5.0,8.0,3660.0,116.0,573.0
4,2024-0339,Brisbane,Hard,32,A,20240101,296,126128,NaN,NaN,...,37.0,27.0,16.0,10.0,5.0,8.0,39.0,1122.0,44.0,1021.0


In [4]:
# Colonnes à harmoniser
cols_jeff = set(df_jeff.columns)
cols_kaggle = set(df_2025.columns)

missing_in_kaggle = cols_jeff - cols_kaggle
extra_in_kaggle = cols_kaggle - cols_jeff

print("Colonnes manquantes dans Kaggle :", missing_in_kaggle)
print("Colonnes supplémentaires dans Kaggle :", extra_in_kaggle)


print("Colonnes dans jeff :", cols_jeff)
print("Colonnes dans Kaggle :", cols_kaggle)

Colonnes manquantes dans Kaggle : {'w_bpFaced', 'winner_seed', 'winner_rank_points', 'winner_rank', 'loser_hand', 'w_bpSaved', 'loser_rank', 'l_svpt', 'tourney_name', 'winner_hand', 'l_bpFaced', 'w_1stWon', 'l_ace', 'l_2ndWon', 'winner_name', 'score', 'winner_ioc', 'loser_ioc', 'loser_entry', 'w_df', 'loser_name', 'w_2ndWon', 'surface', 'match_num', 'w_ace', 'winner_id', 'l_SvGms', 'l_bpSaved', 'l_df', 'loser_id', 'minutes', 'tourney_id', 'winner_age', 'tourney_date', 'l_1stIn', 'tourney_level', 'best_of', 'round', 'w_SvGms', 'draw_size', 'loser_seed', 'loser_rank_points', 'winner_ht', 'loser_ht', 'loser_age', 'l_1stWon', 'winner_entry', 'w_1stIn', 'w_svpt'}
Colonnes supplémentaires dans Kaggle : {'Surface', 'Odd_2', 'Series', 'Player_1', 'Best of', 'Court', 'Rank_1', 'Pts_2', 'Round', 'Odd_1', 'Score', 'Date', 'Tournament', 'Player_2', 'Winner', 'Rank_2', 'Pts_1'}
Colonnes dans jeff : {'w_bpFaced', 'winner_seed', 'loser_hand', 'winner_rank', 'winner_rank_points', 'w_bpSaved', 'loser_r

In [5]:
# 1. Dictionnaire de renommage (Kaggle → Jeff)
rename_cols = {
    "Player_1": "winner_name",
    "Player_2": "loser_name",
    "Rank_1": "winner_rank",
    "Rank_2": "loser_rank",
    "Pts_1": "winner_rank_points",
    "Pts_2": "loser_rank_points",
    "Surface": "surface",
    "Date": "tourney_date",
    "Score": "score",
    "Round": "round",
    "Best of": "best_of",
    "Tournament": "tourney_name"
}

# 2. Appliquer le renommage
df_2025_renamed = df_2025.rename(columns=rename_cols)

# 3. Supprimer les colonnes inutiles de Kaggle (non présentes dans Jeff)
columns_to_drop = set(df_2025_renamed.columns) - set(df_jeff.columns)
df_2025_cleaned = df_2025_renamed.drop(columns=columns_to_drop)

# 4. Ajouter les colonnes manquantes de Jeff (avec NaN ou 0 selon le type)
for col in df_jeff.columns:
    if col not in df_2025_cleaned.columns:
        dtype = df_jeff[col].dtype
        default = 0 if np.issubdtype(dtype, np.number) else np.nan
        df_2025_cleaned[col] = default

# 5. Réordonner les colonnes pour matcher exactement Jeff
df_2025_cleaned = df_2025_cleaned[df_jeff.columns]

# Résultat final
print("Harmonisation terminée.")
print(f"Final shape: {df_2025_cleaned.shape}")
df_2025_cleaned.head(3)


Harmonisation terminée.
Final shape: (958, 49)


,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,...,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
64181,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,69,776,28,1660
64182,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,35,1370,75,730
64183,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,0,0,NaN,...,0,0,0,0,0,0,68,778,10,3350


In [6]:
col = ["winner_name", "winner_id"]

df_2025_cleaned[col]


,winner_name,winner_id
64181,Nishioka Y.,0
64182,Jarry N.,0
64183,Vukic A.,0
64184,Thompson J.,0
64185,Lajovic D.,0
...,...,...
65146,Draper J.,0
65147,Diallo G.,0
65148,Ruud C.,0
65149,Musetti L.,0


In [7]:
import unicodedata
import pandas as pd

# --- 1. Helpers ---

def normalize_name(s):
    """Remove accents, spaces, punctuation, convert to lowercase ASCII."""
    if pd.isna(s):
        return ""
    s = str(s).lower().replace("-", "").replace(".", "").replace(" ", "")
    s = unicodedata.normalize('NFD', s).encode('ascii', 'ignore').decode("utf-8")
    return s.strip()

# --- 2. Load and prepare ATP player reference ---

players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})

# Build normalized full names for fuzzy matching
players["name_full"] = (
    players["name_last"].fillna("").str.strip().str.title() + " " +
    players["name_first"].fillna("").str.strip().str.title()
)
players["name_norm"] = (
    players["name_last"].fillna('') + players["name_first"].fillna('').str[0]
).apply(normalize_name)

# Build short name format "Last F." for first pass
players["short_name"] = (
    players["name_last"].str.strip().str.title() + " " +
    players["name_first"].str.strip().str[0].str.upper()
)

# Dicts for exact and fallback
name_to_id = dict(zip(players["short_name"], players["player_id"]))
norm_to_id = dict(zip(players["name_norm"], players["player_id"]))

# --- 3. Clean 2025 dataset names ---

df_2025_cleaned["winner_last"] = df_2025_cleaned["winner_name"].str.extract(r"^([\w\-']+)")
df_2025_cleaned["winner_initial"] = df_2025_cleaned["winner_name"].str.extract(r"\s+(\w)")
df_2025_cleaned["loser_last"] = df_2025_cleaned["loser_name"].str.extract(r"^([\w\-']+)")
df_2025_cleaned["loser_initial"] = df_2025_cleaned["loser_name"].str.extract(r"\s+(\w)")

df_2025_cleaned["winner_norm"] = (
    df_2025_cleaned["winner_last"].fillna('') + df_2025_cleaned["winner_initial"].fillna('')
).apply(normalize_name)

df_2025_cleaned["loser_norm"] = (
    df_2025_cleaned["loser_last"].fillna('') + df_2025_cleaned["loser_initial"].fillna('')
).apply(normalize_name)

# --- 4. Define fallback fuzzy matching function ---

def fuzzy_find_id(norm, players_df):
    """Try to match based on normalized name being in full name."""
    for _, row in players_df.iterrows():
        if norm in normalize_name(row["name_full"]):
            return row["player_id"]
    return pd.NA

# --- 5. Apply ID resolution with fallback logic ---

# First pass (from earlier mappings if present)
df_2025_cleaned["winner_id"] = df_2025_cleaned["winner_norm"].map(norm_to_id)
df_2025_cleaned["loser_id"]  = df_2025_cleaned["loser_norm"].map(norm_to_id)

# Fallback: fuzzy contains logic only where still missing
df_2025_cleaned["winner_id"] = df_2025_cleaned.apply(
    lambda row: row["winner_id"] if pd.notna(row["winner_id"])
    else fuzzy_find_id(row["winner_norm"], players),
    axis=1
)

df_2025_cleaned["loser_id"] = df_2025_cleaned.apply(
    lambda row: row["loser_id"] if pd.notna(row["loser_id"])
    else fuzzy_find_id(row["loser_norm"], players),
    axis=1
)

# Convert to nullable integer
df_2025_cleaned["winner_id"] = df_2025_cleaned["winner_id"].astype("Int64")
df_2025_cleaned["loser_id"] = df_2025_cleaned["loser_id"].astype("Int64")

# --- 6. Summary ---

missing = (df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()).sum()
total = len(df_2025_cleaned)

print(f"ID resolved: {total - missing} / {total}")
print(f"Still missing: {missing}")

if missing > 0:
    print("\nExamples of unmatched rows:")
    display(df_2025_cleaned[
        df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()
    ][["winner_name", "loser_name"]].head(20))

# --- 7. Manual patch for known unresolved players ---

manual_fixes = {
    "Mpetshi G.": 208659  # Giovanni Mpetshi Perricard
}

for short_name, pid in manual_fixes.items():
    df_2025_cleaned.loc[
        (df_2025_cleaned["winner_name"] == short_name) & (df_2025_cleaned["winner_id"].isna()),
        "winner_id"
    ] = pid
    df_2025_cleaned.loc[
        (df_2025_cleaned["loser_name"] == short_name) & (df_2025_cleaned["loser_id"].isna()),
        "loser_id"
    ] = pid


# Drop temporary columns
df_2025_cleaned.drop(columns=[
    "winner_last", "winner_initial", "loser_last", "loser_initial",
    "winner_norm", "loser_norm"
], inplace=True)


# Final summary
missing = (df_2025_cleaned["winner_id"].isna() | df_2025_cleaned["loser_id"].isna()).sum()
print(f"\nFinal ID resolution: {len(df_2025_cleaned) - missing} / {len(df_2025_cleaned)}")

/tmp/ipykernel_347437/3309772158.py:16: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})


ID resolved: 947 / 958
Still missing: 11

Examples of unmatched rows:


,winner_name,loser_name
64187,Tiafoe F.,Mpetshi G.
64190,Mpetshi G.,Mensik J.
64192,Opelka R.,Mpetshi G.
64313,Mpetshi G.,Monfils G.
64638,Mpetshi G.,Zhang Zh.
64653,Mpetshi G.,Medvedev D.
64752,Griekspoor T.,Mpetshi G.
64833,Mpetshi G.,Thompson J.
64953,Mpetshi G.,Thompson J.
65008,Medjedovic H.,Mpetshi G.



Final ID resolution: 958 / 958


In [8]:
# Print all columns for matches where winner_id == 208659
matches = df_2025_cleaned[df_2025_cleaned["winner_id"] == 208659]

# Ensure all columns are shown
pd.set_option('display.max_columns', None)

# Print the result (with winner_name first for clarity)
cols = ['winner_name'] + [col for col in matches.columns if col != 'winner_name']
print(matches[cols].head())


      winner_name  tourney_id                tourney_name surface  draw_size  \
64190  Mpetshi G.         NaN      Brisbane International    Hard          0   
64313  Mpetshi G.         NaN             Australian Open    Hard          0   
64638  Mpetshi G.         NaN  Dubai Tennis Championships    Hard          0   
64653  Mpetshi G.         NaN  Dubai Tennis Championships    Hard          0   
64833  Mpetshi G.         NaN                  Miami Open    Hard          0   

       tourney_level tourney_date  match_num  winner_id  winner_seed  \
64190            NaN   2025-01-03          0     208659            0   
64313            NaN   2025-01-14          0     208659            0   
64638            NaN   2025-02-24          0     208659            0   
64653            NaN   2025-02-26          0     208659            0   
64833            NaN   2025-03-22          0     208659            0   

       winner_entry  winner_hand  winner_ht  winner_ioc  winner_age  loser_id  \
64190

## Name fiting

In [9]:
# Charger la table des joueurs
players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})

# Créer une colonne fullname (Nom complet = Prénom + Nom)
players["fullname"] = players["name_first"].str.strip().str.title() + " " + players["name_last"].str.strip().str.title()

# Mapper les noms complets dans le dataset principal
id_to_name = dict(zip(players["player_id"], players["fullname"]))

# Remplacer les colonnes winner_name et loser_name par les vrais noms
df_2025_cleaned["winner_name"] = df_2025_cleaned["winner_id"].map(id_to_name)
df_2025_cleaned["loser_name"] = df_2025_cleaned["loser_id"].map(id_to_name)

# Affichage pour vérification
df_2025_cleaned[["winner_id", "winner_name", "loser_id", "loser_name"]].head()

/tmp/ipykernel_347437/605982315.py:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv("../../Data/players/atp_players.csv", dtype={"player_id": int})


,winner_id,winner_name,loser_id,loser_name
64181,132399,Yasuo Nishioka,208103,Jiri Lehecka
64182,111797,Nicolas Jarry,126127,Benjamin Bonzi
64183,126846,Aleksandar Vukic,105777,Grigor Dimitrov
64184,147490,Jerry Thompson,210506,Alex Michelsen
64185,105583,Dusan Lajovic,210150,Jakub Mensik


In [10]:
df_2025_cleaned

,tourney_id,tourney_name,surface,draw_size,tourney_level,tourney_date,match_num,winner_id,winner_seed,winner_entry,winner_name,winner_hand,winner_ht,winner_ioc,winner_age,loser_id,loser_seed,loser_entry,loser_name,loser_hand,loser_ht,loser_ioc,loser_age,score,best_of,round,minutes,w_ace,w_df,w_svpt,w_1stIn,w_1stWon,w_2ndWon,w_SvGms,w_bpSaved,w_bpFaced,l_ace,l_df,l_svpt,l_1stIn,l_1stWon,l_2ndWon,l_SvGms,l_bpSaved,l_bpFaced,winner_rank,winner_rank_points,loser_rank,loser_rank_points
64181,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,132399,0,NaN,Yasuo Nishioka,NaN,0,NaN,0,208103,0,NaN,Jiri Lehecka,NaN,0,NaN,0,2-6 7-6 4-6,3,2nd Round,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,69,776,28,1660
64182,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,111797,0,NaN,Nicolas Jarry,NaN,0,NaN,0,126127,0,NaN,Benjamin Bonzi,NaN,0,NaN,0,6-2 6-3,3,2nd Round,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35,1370,75,730
64183,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,126846,0,NaN,Aleksandar Vukic,NaN,0,NaN,0,105777,0,NaN,Grigor Dimitrov,NaN,0,NaN,0,2-6 6-7,3,2nd Round,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,68,778,10,3350
64184,NaN,Brisbane International,Hard,0,NaN,2025-01-01,0,147490,0,NaN,Jerry Thompson,NaN,0,NaN,0,210506,0,NaN,Alex Michelsen,NaN,0,NaN,0,7-5 6-3,3,2nd Round,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,26,1745,41,1245
64185,NaN,Brisbane International,Hard,0,NaN,2025-01-02,0,105583,0,NaN,Dusan Lajovic,NaN,0,NaN,0,210150,0,NaN,Jakub Mensik,NaN,0,NaN,0,3-6 2-6,3,2nd Round,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,81,710,48,1136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65146,NaN,Mutua Madrid Open,Clay,0,NaN,2025-05-01,0,207733,0,NaN,Jack Draper,NaN,0,NaN,0,208286,0,NaN,Matteo Arnaldi,NaN,0,NaN,0,6-0 6-4,3,Quarterfinals,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,6,3820,44,1240
65147,NaN,Mutua Madrid Open,Clay,0,NaN,2025-05-01,0,209113,0,NaN,Gabriel Diallo,NaN,0,NaN,0,207518,0,NaN,Lorenzo Musetti,NaN,0,NaN,0,4-6 3-6,3,Quarterfinals,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,78,791,11,3200
65148,NaN,Mutua Madrid Open,Clay,0,NaN,2025-05-02,0,134770,0,NaN,Casper Ruud,NaN,0,NaN,0,202103,0,NaN,Francisco Cerundolo,NaN,0,NaN,0,6-4 7-5,3,Semifinals,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,15,2815,21,2225
65149,NaN,Mutua Madrid Open,Clay,0,NaN,2025-05-02,0,207518,0,NaN,Lorenzo Musetti,NaN,0,NaN,0,207733,0,NaN,Jack Draper,NaN,0,NaN,0,3-6 6-7,3,Semifinals,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,11,3200,6,3820


### Tourney date good type

In [11]:
# Harmoniser tourney_date au format int YYYYMMDD
df_2025_cleaned["tourney_date"] = pd.to_datetime(df_2025_cleaned["tourney_date"], errors="coerce")
df_2025_cleaned["tourney_date"] = df_2025_cleaned["tourney_date"].dt.strftime("%Y%m%d").astype(int)  # Drop the '-'

### Save the 2025 ATP dataset

In [12]:
df_2025_cleaned.to_csv("../../Data/atp/atp_matches_2025.csv", index=False)